# Overlay tif image on ipyleaflet

Crashing. To be reviewed...

In [ ]:
! wget https://data.hydrosheds.org/file/hydrosheds-v1-con/au_con_3s/s40e170_con.zip

In [ ]:
! unzip -o s40e170_con.zip

In [ ]:
! gdalinfo s40e170_con.tif

In [ ]:
%%bash

export PROJ_LIB="$CONDA_PREFIX/share/proj"
rm -rf s40e170_con_merc.tif
gdalwarp -s_srs epsg:4326 -t_srs epsg:3857 s40e170_con.tif s40e170_con_merc.tif

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import copy
from base64 import b64encode
import numpy as np
import ipyleaflet

cmap = copy.copy(plt.get_cmap('jet'))
cmap.set_under('k', alpha=0)

def geotiff_to_png(filein, fileout, cmap='jet', vmin=0.1, vmax=2000, mask_value=32767):

    print(f'Reading {filein}')
    src = rasterio.open(filein)
    print(src.crs.to_proj4())
    print(src.bounds)

    data = src.read(1)
    print(data.shape, data.min(), data.max())
    data = np.where(data == mask_value, np.nan, data)
    print(data.shape, np.nanmin(data), np.nanmax(data))

    print(f'Plotting into {fileout}')
    plt.imshow(data, cmap=cmap, aspect='equal', vmin=vmin, vmax=vmax)
    plt.axis('off')
    plt.savefig(fileout, transparent=True, bbox_inches='tight', dpi=500,pad_inches = 0)
    plt.show()
    plt.close()

def get_latlon_gridlines(window=[-180,180,-90,90], dx=10,dy=10):
    x1,x2,y1,y2 = window
    gridlines = []
    # parallels for each lat
    for lat in np.arange(y1, y2+dy/2, dy):
        gridlines += [([lat,x1], [lat,x2])]
    # meridians for each lon
    for lon in  np.arange(x1, x2+dx/2, dx):
        gridlines += [([y1,lon], [y2,lon])]

    return(gridlines)

In [ ]:
geotiff_to_png('s40e170_con_merc.tif', 's40e170_con_merc.png', cmap='jet')

In [ ]:
! gdalinfo s40e170_con.tif

In [ ]:
# bounds = ((miny, minx), (maxy, maxx))
x1,x2,y1,y2 = 170,180,-40,-30
bounds = ((y1, x1), (y2,x2))
print(bounds)

gridlines = get_latlon_gridlines(window=[x1,x2,y1,y2], dx=1,dy=1)


In [ ]:

m = ipyleaflet.Map(center=[-40,170], zoom = 4, scroll_wheel_zoom=True)


m.add_layer(
    ipyleaflet.Polyline(
        name='gridlines',
        locations=gridlines,
        color="gray" ,
        fill=False,
        weight=1
    )
)

with open('s40e170_con_merc.png', "rb") as f:
    data = b64encode(f.read())
    data = data.decode("ascii")
    imgurl = "data:image/png;base64," + data

image = ipyleaflet.ImageOverlay(url=imgurl, bounds=bounds, opacity=0.4, name='merc')
m.add_layer(image)

m.add_control(ipyleaflet.LayersControl())
m
